##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 4 - Fine tuning a custom model

Welcome back to the Kaggle 5-day Generative AI course!

In this notebook you will use the Gemini API to fine-tune a custom, task-specific model. Fine-tuning can be used for a variety of tasks from classic NLP problems like entity extraction or summarisation, to creative tasks like stylised generation. You will fine-tune a model to classify the category a piece of text (a newsgroup post) into the category it belongs to (the newsgroup name).

This codelab walks you tuning a model with the API. [AI Studio](https://aistudio.google.com/app/tune) also supports creating new tuned models directly in the web UI, allowing you to quickly create and monitor models using data from Google Sheets, Drive or your own files.

**Note**: We recommend doing this codelab first today. There may be a period of waiting while the model tunes, so if you start with this one, you can try the other codelab while you wait.

In [5]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

In [6]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [7]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Explore available models

You will be using the [`TunedModel.create`](https://ai.google.dev/api/tuning#method:-tunedmodels.create) API method to start the fine-tuning job and create your custom model. Find a model that supports it through the [`models.list`](https://ai.google.dev/api/models#method:-models.list) endpoint. You can also find more information about tuning models in [the model tuning docs](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?lang=python).

In [13]:
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)

## Download the dataset

In this activity, you will use the same newsgroups dataset that [you used to train a classifier in Keras](https://www.kaggle.com/code/markishere/day-2-classifying-embeddings-with-keras/). In this example you will use a fine-tuned Gemini model to achieve the same goal.

The [20 Newsgroups Text Dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) contains 18,000 newsgroups posts on 20 topics divided into training and test sets.

In [14]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")

# View list of class names for dataset
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Here's what a single row looks like.

In [18]:
print(newsgroups_train.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







## Prepare the dataset

You'll use the same pre-processing code you used for the custom model on day 2. This pre-processing removes personal information, which can be used to "shortcut" to known users of a forum, and formats the text to appear a bit more like regular text and less like a newsgroup post (e.g. by removing the mail headers). This normalisation allows the model to generalise to regular text and not over-depend on specific fields. If your input data is always going to be newsgroup posts, it may be helpful to leave this structure in place if they provide genuine signals.

In [20]:
import email
import re

import pandas as pd


def preprocess_newsgroup_row(data):
    # Extract only the subject and body
    msg = email.message_from_string(data)
    text = f"{msg['Subject']}\n\n{msg.get_payload()}"
    # Strip any remaining email addresses
    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)
    # Truncate the text to fit within the input limits
    text = text[:40000]

    return text


def preprocess_newsgroup_data(newsgroup_dataset):
    # Put data points into dataframe
    df = pd.DataFrame(
        {"Text": newsgroup_dataset.data, "Label": newsgroup_dataset.target}
    )
    # Clean up the text
    df["Text"] = df["Text"].apply(preprocess_newsgroup_row)
    # Match label to target name index
    df["Class Name"] = df["Label"].map(lambda l: newsgroup_dataset.target_names[l])

    return df

In [21]:
# Apply preprocessing to training and test datasets
df_train = preprocess_newsgroup_data(newsgroups_train)
df_test = preprocess_newsgroup_data(newsgroups_test)

df_train.head()

,Text,Label,Class Name
0,WHAT car is this!?\n\n I was wondering if anyo...,7,rec.autos
1,SI Clock Poll - Final Call\n\nA fair number of...,4,comp.sys.mac.hardware
2,"PB questions...\n\nwell folks, my mac plus fin...",4,comp.sys.mac.hardware
3,Re: Weitek P9000 ?\n\nRobert J.C. Kyanko () wr...,1,comp.graphics
4,Re: Shuttle Launch Question\n\nFrom article <>...,14,sci.space


Now sample the data. You will keep 50 rows for each category for training. Note that this is even fewer than the Keras example, as this technique (parameter-efficient fine-tuning, or PEFT) updates a relatively small number of parameters and does not require training a new model or updating the large model.

In [23]:
def sample_data(df, num_samples, classes_to_keep):
    # Sample rows, selecting num_samples of each Label.
    df = (
        df.groupby("Label")[df.columns]
        .apply(lambda x: x.sample(num_samples))
        .reset_index(drop=True)
    )

    df = df[df["Class Name"].str.contains(classes_to_keep)]
    df["Class Name"] = df["Class Name"].astype("category")

    return df


TRAIN_NUM_SAMPLES = 50
TEST_NUM_SAMPLES = 10
# Keep rec.* and sci.*
CLASSES_TO_KEEP = "^rec|^sci"

df_train = sample_data(df_train, TRAIN_NUM_SAMPLES, CLASSES_TO_KEEP)
df_test = sample_data(df_test, TEST_NUM_SAMPLES, CLASSES_TO_KEEP)

## Evaluate baseline performance

Before you start tuning a model, it's good practice to perform an evaluation on the available models to ensure you can measure how much the tuning helps.

First identify a single sample row to use for visual inspection.

In [24]:
sample_idx = 0
sample_row = preprocess_newsgroup_row(newsgroups_test.data[sample_idx])
sample_label = newsgroups_test.target_names[newsgroups_test.target[sample_idx]]

print(sample_row)
print('---')
print('Label:', sample_label)

Need info on 88-89 Bonneville


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler

---
Label: rec.autos


Passing the text directly in as a prompt does not yield the desired results. The model will attempt to respond to the message.

In [25]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=sample_row)
print(response.text)

Hi Neil,

It's understandable you're a little confused! Pontiac did use a somewhat convoluted naming scheme for the Bonneville during that era, and model variations can be subtle. Let's break down the 1988-1989 Bonneville models, their features, and then tackle the pricing questions.

The 1988-1989 Bonneville was built on GM's H-body platform (shared with the Buick LeSabre and Oldsmobile Delta 88). All models for these years came standard with the **3.8L (3800 Series I) V6 engine**, producing around 165 horsepower and 210 lb-ft of torque, mated to a 4-speed automatic transmission (the 4T60). Performance differences between the models are primarily in handling and features, not engine output.

---

### 1988-1989 Bonneville Models & Differences:

Here's a general hierarchy and breakdown of features:

#### 1. Bonneville LE (Luxury Edition)
*   **Position:** This was the base model, focused on comfort and value.
*   **Features:**
    *   Standard cloth interior.
    *   Less equipped dashb

You can use the prompt engineering techniques you have learned this week to induce the model to perform the desired task. Try some of your own ideas and see what is effective, or check out the following cells for different approaches. Note that they have different levels of effectiveness!

In [26]:
# Ask the model directly in a zero-shot prompt.

prompt = "From what newsgroup does the following message originate?"
baseline_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[prompt, sample_row])
print(baseline_response.text)

The message most likely originates from the newsgroup **rec.autos.misc** or a similar automotive-related newsgroup such as **rec.autos**.

`rec.autos.misc` is a general discussion group for various automotive topics that don't fit into more specific `rec.autos` subgroups (like `rec.autos.tech` for technical questions or `rec.autos.marketplace` for buying/selling specific items). This message, asking about models, features, performance, value, and buying advice, fits perfectly within the scope of `rec.autos.misc`.


This technique still produces quite a verbose response. You could try and parse out the relevant text, or refine the prompt even further.

In [27]:
from google.api_core import retry

# You can use a system instruction to do more direct prompting, and get a
# more succinct answer.

system_instruct = """
You are a classification service. You will be passed input that represents
a newsgroup post and you must respond with the newsgroup from which the post
originates.
"""

# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# If you want to evaluate your own technique, replace this body of this function
# with your model, prompt and other code and return the predicted answer.
@retry.Retry(predicate=is_retriable)
def predict_label(post: str) -> str:
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(
            system_instruction=system_instruct),
        contents=post)

    rc = response.candidates[0]

    # Any errors, filters, recitation, etc we can mark as a general error
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        # Clean up the response.
        return response.text.strip()


prediction = predict_label(sample_row)

print(prediction)
print()
print("Correct!" if prediction == sample_label else "Incorrect.")

rec.autos

Correct!


Now run a short evaluation using the function defined above. The test set is further sampled to ensure the experiment runs smoothly on the API's free tier. In practice you would evaluate over the whole set.

In [28]:
import tqdm
from tqdm.rich import tqdm as tqdmr
import warnings

# Enable tqdm features on Pandas.
tqdmr.pandas()

# But suppress the experimental warning
warnings.filterwarnings("ignore", category=tqdm.TqdmExperimentalWarning)


# Further sample the test data to be mindful of the free-tier quota.
df_baseline_eval = sample_data(df_test, 2, '.*')

# Make predictions using the sampled data.
df_baseline_eval['Prediction'] = df_baseline_eval['Text'].progress_apply(predict_label)

# And calculate the accuracy.
accuracy = (df_baseline_eval["Class Name"] == df_baseline_eval["Prediction"]).sum() / len(df_baseline_eval)
print(f"Accuracy: {accuracy:.2%}")

Output()

Accuracy: 37.50%


Now take a look at the dataframe to compare the predictions with the labels.

In [29]:
df_baseline_eval

,Text,Label,Class Name,Prediction
0,VASCAR\n\nI know this is the wrong place to po...,7,rec.autos,soc.motss
1,Re: Mercury Villager Minivan -- good buy?\n\nI...,7,rec.autos,rec.autos
2,"Re: dogs\n\nIn article <>, (Charles Parr) wri...",8,rec.motorcycles,`rec.pets.dogs`
3,Re: DoD Pins...NOT!\n\nIn article <> writes:\...,8,rec.motorcycles,rec.motorcycles
4,Re: Neon Deon Sanders (Braves & Giants)\n\nIn...,9,rec.sport.baseball,rec.sport.baseball
5,Mea Culpa -- Bosio no-no\n\nLike Clinton and R...,9,rec.sport.baseball,`rec.sport.baseball`
6,Re: Grant Fuhr leads Sabres\n\nIn <> \n(Deepak...,10,rec.sport.hockey,rec.sport.hockey
7,"Buffalo Sabres\n\n Ok, Buffalo fans. I am a ...",10,rec.sport.hockey,rec.sport.hockey
8,Re: PGP Where to get it?\n\n>At the moment PGP...,11,sci.crypt,comp.security.pgp
9,Re: Wiretapping reality today\n\n-----BEGIN PG...,11,sci.crypt,soc.politics


## Tune a custom model

In this example you'll use tuning to create a model that requires no prompting or system instructions and outputs succinct text from the classes you provide in the training data.

The data contains both input text (the processed posts) and output text (the category, or newsgroup), that you can use to start tuning a model.

When calling `tune()`, you can specify model tuning hyperparameters too:
 - `epoch_count`: defines how many times to loop through the data,
 - `batch_size`: defines how many rows to process in a single step, and
 - `learning_rate`: defines the scaling factor for updating model weights at each step.

You can also choose to omit them and use the defaults. [Learn more](https://developers.google.com/machine-learning/crash-course/linear-regression/hyperparameters) about these parameters and how they work. For this example these parameters were selected by running some tuning jobs and selecting parameters that converged efficiently.

This example will start a new tuning job, but only if one does not already exist. This allows you to leave this codelab and come back later - re-running this step will find your last model.

In [32]:
### from collections.abc import Iterable
## import random


# Convert the data frame into a dataset suitable for tuning.
## input_data = {'examples': 
##    df_train[['Text', 'Class Name']]
##.rename(columns={'Text': 'textInput', 'Class Name': 'output'})
##      .to_dict(orient='records')
## }

# If you are re-running this lab, add your model_id here.
##model_id = None

# # Or try and find a recent tuning job.
# if not model_id:
#   queued_model = None
#   # Newest models first.
#   for m in reversed(client.tunings.list()):
#     # Only look at newsgroup classification models.
#     if m.name.startswith('tunedModels/newsgroup-classification-model'):
#       # If there is a completed model, use the first (newest) one.
#       if m.state.name == 'JOB_STATE_SUCCEEDED':
#         model_id = m.name
#         print('Found existing tuned model to reuse.')
#         break

#       elif m.state.name == 'JOB_STATE_RUNNING' and not queued_model:
#         # If there's a model still queued, remember the most recent one.
#         queued_model = m.name
#   else:
#     if queued_model:
#       model_id = queued_model
#       print('Found queued model, still waiting.')


# # Upload the training data and queue the tuning job.
# if not model_id:
#     tuning_op = client.tunings.tune(
#         base_model="models/gemini-1.5-flash-001-tuning",
#         training_dataset=input_data,
#         config=types.CreateTuningJobConfig(
#             tuned_model_display_name="Newsgroup classification model",
#             batch_size=16,
#             epoch_count=2,
#         ),
#     )

#     print(tuning_op.state)
#     model_id = tuning_op.name

# print(model_id)

In [41]:
encontrados = []

# Listamos todos os modelos e filtramos pela permissão de tuning
for m in client.models.list():
    # Verificamos se 'createTunedModel' está nas ações suportadas
    if any('createTunedModel' in action or 'create_tuned_model' in action for action in m.supported_actions):
        print(f"✅ MODELO ENCONTRADO: {m.name}")
        encontrados.append(m.name)

if not encontrados:
    print("❌ Nenhum modelo de tuning disponível para esta API Key.")
    print("Verifique se o faturamento (billing) está ativo no console do Google Cloud vinculado.")

❌ Nenhum modelo de tuning disponível para esta API Key.
Verifique se o faturamento (billing) está ativo no console do Google Cloud vinculado.


In [42]:
compatíveis = []

for m in client.models.list():
    # No novo SDK, a propriedade pode variar, vamos checar 'create_tuned_model'
    if 'createTunedModel' in m.supported_actions or 'create_tuned_model' in str(m.supported_actions).lower():
        print(f"✅ DISPONÍVEL: {m.name}")
        compatíveis.append(m.name)

if not compatíveis:
    print("❌ Nenhum modelo de tuning encontrado na sua região/projeto.")
    print("Dica: Tente usar 'models/gemini-1.5-flash-001' diretamente.")

❌ Nenhum modelo de tuning encontrado na sua região/projeto.
Dica: Tente usar 'models/gemini-1.5-flash-001' diretamente.


### NÃO HÁ MAIS MODELOS DE TUNING DISPONÍVEIS PELO AI STUDIO

Com a descontinuação do Gemini 1.5 Flash-001 em maio de 2025, não teremos mais um modelo disponível que ofereça suporte ao ajuste fino na API Gemini ou no AI Studio, mas ele é compatível com a Vertex AI.

Planejamos trazer de volta o suporte ao ajuste refinado no futuro. Gostaríamos de receber sua opinião no fórum de desenvolvedores se o ajuste fino for importante para seu caso de uso.

In [23]:
for model in client.models.list():
    print(f"Nome: {model.name} - Suporta Tuning: {model.supported_actions}")

Nome: models/gemini-2.5-flash - Suporta Tuning: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
Nome: models/gemini-2.5-pro - Suporta Tuning: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
Nome: models/gemini-2.0-flash - Suporta Tuning: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
Nome: models/gemini-2.0-flash-001 - Suporta Tuning: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
Nome: models/gemini-2.0-flash-exp-image-generation - Suporta Tuning: ['generateContent', 'countTokens', 'bidiGenerateContent']
Nome: models/gemini-2.0-flash-lite-001 - Suporta Tuning: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
Nome: models/gemini-2.0-flash-lite - Suporta Tuning: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
Nome: models/gemini-2.5-flash-preview-tts - Suporta Tuning: ['countTokens', '

This has created a tuning job that will run in the background. To inspect the progress of the tuning job, run this cell to plot the current status and loss curve. Once the status reaches `ACTIVE`, tuning is complete and the model is ready to use.

Tuning jobs are queued, so it may look like no training steps have been taken initially but it will progress. Tuning can take anywhere from a few minutes to multiple hours, depending on factors like your dataset size and how busy the tuning infrastrature is. Why not treat yourself to a nice cup of tea while you wait, or come and say "Hi!" in the group [Discord](https://discord.com/invite/kaggle).

It is safe to stop this cell at any point. It will not stop the tuning job.

**IMPORTANT**: Due to the high volume of users doing this course, tuning jobs may be queued for many hours. Take a note of your tuned model ID above (`tunedModels/...`) so you can come back to it tomorrow. In the meantime, check out the [Search grounding](https://www.kaggle.com/code/markishere/day-4-google-search-grounding/) codelab. If you want to try tuning a local LLM, check out [the fine-tuning guides for tuning a Gemma model](https://ai.google.dev/gemma/docs/tune).

In [ ]:
import datetime
import time


MAX_WAIT = datetime.timedelta(minutes=10)

while not (tuned_model := client.tunings.get(name=model_id)).has_ended:

    print(tuned_model.state)
    time.sleep(60)

    # Don't wait too long. Use a public model if this is going to take a while.
    if datetime.datetime.now(datetime.timezone.utc) - tuned_model.create_time > MAX_WAIT:
        print("Taking a shortcut, using a previously prepared model.")
        model_id = "tunedModels/newsgroup-classification-model-ltenbi1b"
        tuned_model = client.tunings.get(name=model_id)
        break


print(f"Done! The model state is: {tuned_model.state.name}")

if not tuned_model.has_succeeded and tuned_model.error:
    print("Error:", tuned_model.error)

## Use the new model

Now that you have a tuned model, try it out with custom data. You use the same API as a normal Gemini API interaction, but you specify your new model as the model name, which will start with `tunedModels/`.

In [ ]:
new_text = """
First-timer looking to get out of here.

Hi, I'm writing about my interest in travelling to the outer limits!

What kind of craft can I buy? What is easiest to access from this 3rd rock?

Let me know how to do that please.
"""

response = client.models.generate_content(
    model=model_id, contents=new_text)

print(response.text)

### Evaluation

You can see that the model outputs labels that correspond to those in the training data, and without any system instructions or prompting, which is already a great improvement. Now see how well it performs on the test set.

Note that there is no parallelism in this example; classifying the test sub-set will take a few minutes.

In [ ]:
@retry.Retry(predicate=is_retriable)
def classify_text(text: str) -> str:
    """Classify the provided text into a known newsgroup."""
    response = client.models.generate_content(
        model=model_id, contents=text)
    rc = response.candidates[0]

    # Any errors, filters, recitation, etc we can mark as a general error
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        return rc.content.parts[0].text


# The sampling here is just to minimise your quota usage. If you can, you should
# evaluate the whole test set with `df_model_eval = df_test.copy()`.
df_model_eval = sample_data(df_test, 4, '.*')

df_model_eval["Prediction"] = df_model_eval["Text"].progress_apply(classify_text)

accuracy = (df_model_eval["Class Name"] == df_model_eval["Prediction"]).sum() / len(df_model_eval)
print(f"Accuracy: {accuracy:.2%}")

## Compare token usage

AI Studio and the Gemini API provide model tuning at no cost, however normal limits and charges apply for *use* of a tuned model.

The size of the input prompt and other generation config like system instructions, as well as the number of generated output tokens, all contribute to the overall cost of a request.

In [ ]:
# Calculate the *input* cost of the baseline model with system instructions.
sysint_tokens = client.models.count_tokens(
    model='gemini-1.5-flash-001', contents=[system_instruct, sample_row]
).total_tokens
print(f'System instructed baseline model: {sysint_tokens} (input)')

# Calculate the input cost of the tuned model.
tuned_tokens = client.models.count_tokens(model=tuned_model.base_model, contents=sample_row).total_tokens
print(f'Tuned model: {tuned_tokens} (input)')

savings = (sysint_tokens - tuned_tokens) / tuned_tokens
print(f'Token savings: {savings:.2%}')  # Note that this is only n=1.

The earlier verbose model also produced more output tokens than needed for this task.

In [ ]:
baseline_token_output = baseline_response.usage_metadata.candidates_token_count
print('Baseline (verbose) output tokens:', baseline_token_output)

tuned_model_output = client.models.generate_content(
    model=model_id, contents=sample_row)
tuned_tokens_output = tuned_model_output.usage_metadata.candidates_token_count
print('Tuned output tokens:', tuned_tokens_output)

## Next steps

Now that you have tuned a classification model, try some other tasks, like tuning a model to respond with a specific tone or style using hand-written examples (or even generated examples!). Kaggle hosts [a number of datasets](https://www.kaggle.com/datasets) you can try out.

Learn about [when supervised fine-tuning is most effective](https://cloud.google.com/blog/products/ai-machine-learning/supervised-fine-tuning-for-gemini-llm).

And check out the [fine-tuning tutorial](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?hl=en&lang=python) for another example that shows a tuned model extending beyond the training data to new, unseen inputs.

*- [Mark McD](https://linktr.ee/markmcd)*